# Infrastructure Multiobjective Example
Created by Alex Dowling (adowling@nd.edu).
Last updated March 1, 2018

Model adapted from Dowling, Ruiz-Mercado, Zavala (2016), Computers & Chemical Engineering.

In [157]:
# Load libraries
using JuMP
using Cbc
using PyPlot

## Specify Input Data

In [158]:
# Seed the random number generator.
srand(1) # If you comment out this line, the results will be different each time the code runs.

# Farms
nFarms = 22
farmLoc = rand(nFarms,2)

# Urban Centers
nUrban = 4
cityLoc = rand(nUrban,2)

# Lakes
nWater = 6
waterLoc = rand(nWater,2)

# Candidate locations for processing sites
nCand = 30
candLoc = rand(nCand,2)

# Specify data for facilities (weights)
# First column: small facility
# Second column: large facility
capacityW = [2, 10] # maximum processing capacity
costW = [2, 8.5] # cost
waterW = [2, 2] # impact on water quality
safetyW = [2, 2]; # impact on safety

## Calculate Distances

In [159]:
function calculateDistanceMatrix(landmarks,candidates)
    n = size(landmarks,1)
    m = size(candidates,1)

    # Initialize distance matrix with zeros
    D = zeros(n,m)

    # Loop over all combinations of landmarks and candidate locations, calculate distance
    for i = 1:n
        for j = 1:m
            D[i,j] = norm(landmarks[i,:] - candidates[j,:])
        end
    end

    # Return distance matrix
    return D
end

dFarmCand = calculateDistanceMatrix(farmLoc, candLoc)
dCityCand = calculateDistanceMatrix(cityLoc, candLoc)
dWaterCand = calculateDistanceMatrix(waterLoc,candLoc);

## Define Optimization Model

In [160]:
# Initialize model
m = Model()

# Define sets
F = 1:nFarms
U = 1:nUrban
W = 1:nWater
C = 1:nCand

# Define four objectives
@variable(m, f[1:4]);

### Discrete Decisions

In [161]:
# Build a SMALL facility at a candidate site? (binary)
@variable(m, ySmall[C], Bin)

# Build a LARGE facility at a candidate site? (binary)
@variable(m, yLarge[C], Bin);

# Only build one facility per candidate location
@constraint(m, OnePerLocation[i=C], ySmall[i] + yLarge[i] <= 1);

### Transportation network and cost (total distance)

In [162]:
# The matrix 'net' models the flow of matrial in the transportation network.
# Rows are farms (sources), columns are candidate facility locations (sinks)
@variable(m, 0 <= net[F,C] <= 1)

# Each farm produces 1 unit of waste
@constraint(m, NetworkIn[i=F], sum(net[i,j] for j=C) == 1)
                
# Each facility cannot process more than its maximum capacity
@constraint(m, NetworkOut[j=C], sum(net[i,j] for i=F) <= capacityW[1]*ySmall[j] + capacityW[2]*yLarge[j])

# Calculate total distance from each farm to its assigned facility
@constraint(m, f[1] == sum(dFarmCand[i,j]*net[i,j] for i=F for j=C));


### Safety metric (distance)

In [163]:
# For each urban center, calculate the distance to the NEAREST facility
@variable(m, minDistUtoC[U] >= 0)

# Big-M value, used for relaxation
M = sqrt(2)

# Calculate distance to NEAREST facility
@constraint(m, CalcMinDistUtoC[i=U, j=C], minDistUtoC[i] <= dCityCand[i,j]*(safetyW[1]*ySmall[j] + safetyW[2]*yLarge[j]) + M*maximum(safetyW)*(1 - ySmall[j] - yLarge[j]))

# Calculate total distance for each urban center to nearest facility
@constraint(m, f[2] == sum(minDistUtoC[i] for i=U));

### Water quality (distance)

In [164]:
# For each water body, calculate the distance to the NEAREST facility
@variable(m, minDistWtoC[W] >= 0)

# Big-M value, used for relaxation
M = sqrt(2)

# Calculate distance to NEAREST facility
@constraint(m, CalcMinDistWtoC[i=W, j=C], minDistWtoC[i] <= dWaterCand[i,j]*(waterW[1]*ySmall[j] + waterW[2]*yLarge[j]) + M*maximum(waterW)*(1 - ySmall[j] - yLarge[j]))

# Calculate total distance for each water body to nearest facility
@constraint(m, f[3] == sum(minDistWtoC[i] for i=W));

### Investment Cost

In [165]:
# Calculate total cost
@constraint(m, f[4] == sum(ySmall[i]*costW[1] + yLarge[i]*costW[2] for i=C));

f[4] - 2 ySmall[1] - 8.5 yLarge[1] - 2 ySmall[2] - 8.5 yLarge[2] - 2 ySmall[3] - 8.5 yLarge[3] - 2 ySmall[4] - 8.5 yLarge[4] - 2 ySmall[5] - 8.5 yLarge[5] - 2 ySmall[6] - 8.5 yLarge[6] - 2 ySmall[7] - 8.5 yLarge[7] - 2 ySmall[8] - 8.5 yLarge[8] - 2 ySmall[9] - 8.5 yLarge[9] - 2 ySmall[10] - 8.5 yLarge[10] - 2 ySmall[11] - 8.5 yLarge[11] - 2 ySmall[12] - 8.5 yLarge[12] - 2 ySmall[13] - 8.5 yLarge[13] - 2 ySmall[14] - 8.5 yLarge[14] - 2 ySmall[15] - 8.5 yLarge[15] - 2 ySmall[16] - 8.5 yLarge[16] - 2 ySmall[17] - 8.5 yLarge[17] - 2 ySmall[18] - 8.5 yLarge[18] - 2 ySmall[19] - 8.5 yLarge[19] - 2 ySmall[20] - 8.5 yLarge[20] - 2 ySmall[21] - 8.5 yLarge[21] - 2 ySmall[22] - 8.5 yLarge[22] - 2 ySmall[23] - 8.5 yLarge[23] - 2 ySmall[24] - 8.5 yLarge[24] - 2 ySmall[25] - 8.5 yLarge[25] - 2 ySmall[26] - 8.5 yLarge[26] - 2 ySmall[27] - 8.5 yLarge[27] - 2 ySmall[28] - 8.5 yLarge[28] - 2 ySmall[29] - 8.5 yLarge[29] - 2 ySmall[30] - 8.5 yLarge[30] = 0

### Specify Objective

In [166]:
# Specify weights for each objective (convert to common units)\
wobj = zeros(4)

# Transportation cost
wobj[1] = 0.1

# Safety (distance between facilities and urban centers)
wobj[2] = -0.2

# Water quality (distance between facilities and body of water)
wobj[3] = -0.3

# Investment cost
wobj[4] = 1

@objective(m, Min, sum(wobj[i]*f[i] for i = 1:4));

## Solve and Visualize Results

### Inspect optimization model

In [ ]:
m

### Solve optimization problem using **Cbc** (open source)

In [ ]:
setsolver(m, CbcSolver())
solve(m)

In [ ]:
println("Objectives: ")
println(getvalue(f))

### Visualize solution using PyPlot

In [ ]:
figure

# Plot candidate locations
scatter(candLoc[:,1], candLoc[:,2],label="Candidates",color="grey",marker=".")

# Visualize waste transportation network
net_sln = getvalue(net)
for i = F
    for j = C
        if(net_sln[i,j] > 0.001)
            x = [farmLoc[i,1], candLoc[j,1]]
            y = [farmLoc[i,2], candLoc[j,2]]
            plot(x, y, color="black", linestyle="--",linewidth=1)
        end
    end
end

# Plot lakes
scatter(waterLoc[:,1], waterLoc[:,2],label="Lakes",color="blue",marker="o")

# Plot cities
scatter(cityLoc[:,1],cityLoc[:,2],label="Cities",color="green",marker="s")

# Plot farms
scatter(farmLoc[:,1],farmLoc[:,2],label="Farms",color="red",marker="^")

# Plot large facilities
yLarge_sln = getvalue(yLarge)
ii = []
for i = C
    if(yLarge_sln[i] > 0.999)
        append!(ii, i)
    end
end
scatter(candLoc[ii,1], candLoc[ii,2],label="Large",color="black",marker="*",s=100)

# Plot small facilities
ySmall_sln = getvalue(ySmall)
ii = []
for i = C
    if(ySmall_sln[i] > 0.999)
        append!(ii, i)
    end
end
scatter(candLoc[ii,1], candLoc[ii,2],label="Small",color="black",marker="x",s=50)

# Add legend
legend(loc="center left", bbox_to_anchor=(1.05, 0.5))

## Activity

Adjust the objective weights and resolve to see how the solution changes.